In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#2. specify parameters
pipeline_params={
}
step_params={
}
substep_params={   
}

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, default_param_values, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(
    
    tmp_inputs =
    [
        { ENTITY_NAME: "cache_data" }
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

![interface 1_export_onnx_interface.drawio](./imgs/1_export_onnx_interface.drawio.png)

In [ ]:
#4 get substep.interface
tmp_inputs = substep.tmp_inputs()

print(f"{tmp_inputs.cache_data=}")

In [ ]:
#5 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

### Reading basic parameters from the config

In [ ]:
# GET config params
import json
import os
import os.path as osp

config_fn = osp.join(tmp_inputs.cache_data, 'config.json')

with open(config_fn) as f_id:
    CONFIG = json.load(f_id)

MODEL_NAME = CONFIG["train_config_parameters"]["MODEL_NAME"]
WORK_DIR = CONFIG['work_dir'] 
ONNX_DIR = CONFIG["onnx_dir"]
CONFIG_DEPLOY = CONFIG["config_deploy"]
DEVICE = CONFIG["device"]
CONFIG_MODEL = CONFIG["config_model"]
TORCH_MODEL = CONFIG["torch_model"]
ONNX_MODEL = CONFIG["onnx_model"]
TYPE_EXPORT = CONFIG["type_export"]
test_image_path = CONFIG["test_image_path"]

TYPE_EXPORT

In [ ]:
from mmdet.datasets import PIPELINES

try:
    @PIPELINES.register_module()
    class DataAsList:
        def __call__(self, results):
            aug_data_dict = {key: [val] for key, val in results.items()}
            return aug_data_dict
except Exception as e:
    print(e)

### Exporting a model to ONNX format
TYPE_EXPORT == "mmdeploy_export" - export the model to onnx format via mmdeploy (post-processing is added to the model graph itself)
TYPE_EXPORT == "torch_export" - export the model to onnx format via torch.export (post-processing is written separately)

In [ ]:
if TYPE_EXPORT == "mmdeploy_export":
    from mmdeploy.apis import torch2onnx
    import numpy as np
    output_file = os.path.join(ONNX_DIR, ONNX_MODEL)

    torch2onnx(img=np.zeros((640,640,3)),
               work_dir=ONNX_DIR,
               save_file=ONNX_MODEL,
               deploy_cfg=CONFIG_DEPLOY,
               model_cfg=CONFIG_MODEL,
               model_checkpoint=TORCH_MODEL,
               device=DEVICE)

In [ ]:
if TYPE_EXPORT == "torch_export":
    from mmcv import Config, ConfigDict
    from mmdet.apis import init_detector, inference_detector, show_result_pyplot
    from mmdet.core.export import preprocess_example_input
    import torch
    
    cfg_deploy = Config.fromfile(CONFIG_DEPLOY)
    cfg_model = Config.fromfile(CONFIG_MODEL)
    
    model = init_detector(CONFIG_MODEL, 
                          TORCH_MODEL, 
                          device=DEVICE)
    
    input_names = cfg_deploy.onnx_config.input_names
    opset_version = cfg_deploy.onnx_config.opset_version
    dynamic_axes = cfg_deploy.onnx_config.dynamic_axes
    # output_names = ['dets', 'labels']
    output_file = os.path.join(ONNX_DIR, ONNX_MODEL)

    from mmcv.parallel import collate
    from mmdet.datasets.pipelines import Compose

    test_pipeline = Compose(cfg_model.data.test.pipeline)
    datas = []
    data = dict(img_info=dict(filename=test_image_path), img_prefix=None)
    data = test_pipeline(data)
    datas.append(data)

    data = collate(datas, samples_per_gpu=1)
    data['img_metas'] = [img_metas.data[0] for img_metas in data['img_metas']]
    data['imgs'] = data.pop('img')
    data['imgs'] = [img.data[0].to(DEVICE) for img in data['imgs']]
    
    model.forward = model.forward_dummy

    torch.onnx.export(
            model,
            data["imgs"][0],
            output_file,
            input_names=input_names,
            # output_names=output_names,
            export_params=True,
            keep_initializers_as_inputs=False,
            training= torch.onnx.TrainingMode.EVAL,
            do_constant_folding=True,
            verbose=False,
            opset_version=opset_version,
            dynamic_axes=dynamic_axes)

    print(output_file)

In [ ]:
CONFIG['out_onnx_model'] = output_file

with open(config_fn, 'w') as f_id:
    json.dump(CONFIG, f_id, indent=4)

In [ ]:
#stop spark
SinaraSpark.stop_session()